### Enhancing LLM Observability with Llama Trace by Arize Phoenix
https://arize.com/docs/phoenix/tracing/llm-traces

*Arize Phoenix is an open-sourced LLM observability tool designed for experimentation, evaluation, and troubleshooting of AI and LLM applications. It allows AI engineers and data scientists to quickly visualize their data, evaluate performance, track down issues, and export data to improve.*

*"Phoenix uses **OpenTelemetry Protocol** to record the paths taken by requests as they propagate through multiple steps or components of an LLM application pipeline"*

*"For example: When a user interacts with an LLM application, tracing can capture the sequence of operations, such as document retrieval, embedding generation, language model invocation, and response generation to provide a detailed timeline of the request's execution."*

In [ ]:
import os

os.environ["PHOENIX_API_KEY"] = "ADD YOUR PHOENIX API KEY"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "ADD YOUR PHOENIX HOSTNAME"

# If you created your Phoenix Cloud instance before June 24th, 2025,
# you also need to set the API key as a header
#os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.getenv('PHOENIX_API_KEY')}"


In [2]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="facinating-things", # Default is 'default'
  auto_instrument=True, # See 'Trace all calls made to a library' below
)
tracer = tracer_provider.get_tracer(__name__)

/Users/jinkettyee/.pyenv/versions/facinating-things/lib/python3.12/site-packages/phoenix/otel/otel.py:333: UserWarning: Could not infer collector endpoint protocol, defaulting to HTTP.
  warnings.warn("Could not infer collector endpoint protocol, defaulting to HTTP.")
DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: facinating-things
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/s/fascinating-things/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [3]:
# Trial simple ReAct Agent w/o tools. 

from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import AgentStream

llm = OpenAI(model="gpt-4o-mini")
agent = ReActAgent(llm=llm)

# Create a context to store the conversation history/session state
ctx = Context(agent)

In [4]:
from llama_index.core.agent.workflow import AgentStream

handler = agent.run("Give an overview of Harry Potter and the Order of the Phoenix.", ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, AgentStream):
        print(f"{ev.delta}", end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I can provide an overview of "Harry Potter and the Order of the Phoenix" without needing to use a tool.
Answer: "Harry Potter and the Order of the Phoenix" is the fifth book in J.K. Rowling's Harry Potter series. The story follows Harry Potter during his fifth year at Hogwarts School of Witchcraft and Wizardry. 

As the wizarding world faces the return of Lord Voldemort, the Ministry of Magic refuses to acknowledge the threat, leading to a rise in tension and fear. Harry, along with his friends Hermione Granger and Ron Weasley, forms a secret group called Dumbledore's Army to prepare students for the challenges ahead, as the new Defense Against the Dark Arts teacher, Dolores Umbridge, imposes strict rules and refuses to teach practical defense skills.

Throughout the book, Harry struggles with the emotional turmoil of adolescence, the loss of loved ones, and the burden of being the Chosen One. The climax occurs during a battle at t

### Define and Run Workflow for Function Agent
- Define workflow events
- Define workflow itself

In [6]:
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import ToolSelection, ToolOutput
from llama_index.core.workflow import Event


class InputEvent(Event):
    input: list[ChatMessage]


class StreamEvent(Event):
    delta: str


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

In [7]:
from typing import Any, List

from llama_index.core.llms.function_calling import FunctionCallingLLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.llms.openai import OpenAI


class FunctionCallingAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: FunctionCallingLLM | None = None,
        tools: List[BaseTool] | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []

        self.llm = llm or OpenAI()
        assert self.llm.metadata.is_function_calling_model

    @step
    async def prepare_chat_history(
        self, ctx: Context, ev: StartEvent
    ) -> InputEvent:
        # clear sources
        await ctx.store.set("sources", [])

        # check if memory is setup
        memory = await ctx.store.get("memory", default=None)
        if not memory:
            memory = ChatMemoryBuffer.from_defaults(llm=self.llm)

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        memory.put(user_msg)

        # get chat history
        chat_history = memory.get()

        # update context
        await ctx.store.set("memory", memory)

        return InputEvent(input=chat_history)

    @step
    async def handle_llm_input(
        self, ctx: Context, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input

        # stream the response
        response_stream = await self.llm.astream_chat_with_tools(
            self.tools, chat_history=chat_history
        )
        async for response in response_stream:
            ctx.write_event_to_stream(StreamEvent(delta=response.delta or ""))

        # save the final response, which should have all content
        memory = await ctx.store.get("memory")
        memory.put(response.message)
        await ctx.store.set("memory", memory)

        # get tool calls
        tool_calls = self.llm.get_tool_calls_from_response(
            response, error_on_no_tool_call=False
        )

        if not tool_calls:
            sources = await ctx.store.get("sources", default=[])
            return StopEvent(
                result={"response": response, "sources": [*sources]}
            )
        else:
            return ToolCallEvent(tool_calls=tool_calls)

    @step
    async def handle_tool_calls(
        self, ctx: Context, ev: ToolCallEvent
    ) -> InputEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}

        tool_msgs = []
        sources = await ctx.store.get("sources", default=[])

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            additional_kwargs = {
                "tool_call_id": tool_call.tool_id,
                "name": tool.metadata.get_name(),
            }
            if not tool:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Tool {tool_call.tool_name} does not exist",
                        additional_kwargs=additional_kwargs,
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                sources.append(tool_output)
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=tool_output.content,
                        additional_kwargs=additional_kwargs,
                    )
                )
            except Exception as e:
                tool_msgs.append(
                    ChatMessage(
                        role="tool",
                        content=f"Encountered error in tool call: {e}",
                        additional_kwargs=additional_kwargs,
                    )
                )

        # update memory
        memory = await ctx.store.get("memory")
        for msg in tool_msgs:
            memory.put(msg)

        await ctx.store.set("sources", sources)
        await ctx.store.set("memory", memory)

        chat_history = memory.get()
        return InputEvent(input=chat_history)

#### Notes: 
- OpenAI models like "GPT-4o-mini" are function calling models which returns *response.message.additional_kwargs["tool_calls"]* - a structured list of tool call instructions if the model chose to invoke tools
- *get_tool_calls_from_response()* method reads this "tool_calls" array and converts each entry into *ToolSelection(...)* object, extracting metadata like tool_id, tool_name and tool_kwargs.
- If there are no tool call instructions, it returns an empty list and hence, the loop goes straight into a StopEvent. 

#### Define query engine and run.

In [ ]:
# import dependencies
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    Settings
)

import qdrant_client
from IPython.display import Markdown, display
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.chat_engine import SimpleChatEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.embeddings.fastembed import FastEmbedEmbedding

/Users/jinkettyee/.pyenv/versions/facinating-things/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# load documents - "../data" folder
docs = SimpleDirectoryReader("../data/books").load_data(show_progress=True)

# build vector store index - Sync & Async
client = qdrant_client.QdrantClient(host='localhost', port=6333)
aclient = qdrant_client.AsyncQdrantClient(host="localhost", port=6333)

# Set LLM & embedding model
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1, max_tokens=1024, streaming=True)
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Initialize Qdrant vector store
vector_store = QdrantVectorStore(
    "interview_notes",
    client=client, 
    aclient = aclient,
    enable_hybrid = True,
    fastembed_sparse_model="Qdrant/bm25"
    )

# Create storage context container
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Build vector store index -> Query Engine
index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    embed_model=Settings.embed_model
)

Loading files:   0%|          | 0/1 [00:00<?, ?it/s]

Loading files: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


In [10]:
# get example response - retrieve 2 sparse, 2 dense, and filter down to 3 total hybrid results
query_engine = index.as_query_engine(
    similarity_top_k=2, 
    sparse_top_k=2, 
    hybrid_top_k=3,
    vector_store_query_mode="hybrid", 
    llm=Settings.llm, 
    # use_async=True,
)
response = query_engine.query(
    "When Dumbledore is wrong, should Harry say that he is wrong? Why or why not?."
    )

In [34]:
display(Markdown(f"<b>{response}</b>"))

<b>It would be more effective for Harry to approach the situation with humility and openness rather than directly telling Dumbledore he is wrong. By expressing uncertainty and inviting a discussion, Harry can create a more constructive dialogue. This approach respects Dumbledore's intelligence and authority, reducing the likelihood of defensiveness and fostering a collaborative examination of the facts.</b>

#### Run FunctionCalling Agent with Query Engine as tool

In [11]:
tools = [
    QueryEngineTool.from_defaults(
        query_engine=query_engine,
        name="Resource",
        description=("Provides information on fundamental techniques in handling people."
                     "Use a detailed plain text question as input to the tool."
        ),
    ),
]

book_agent = FunctionCallingAgent(
    tools=tools,
    llm = Settings.llm,
    timeout=120,
    verbose = False,
)

> *Stream response.*

In [12]:
# alternative queries
alt_query = "Harry wants approval to go to hogsmeade from Dumbledore. He knows that Dumbledore likes Voldemort, design a hypothetical scenario on how he should communicate his wants with principles from the resource."

handler = book_agent.run(input="With principles from the given resource, how should Harry respond to Dumbledore if he found that the latter said something wrong?")

async for event in handler.stream_events():
    if isinstance(event, StreamEvent):
        print(event.delta, end="", flush=True)

response = await handler

Harry should respond to Dumbledore with humility and openness. He might say something like, "Well, I thought otherwise, but I may be wrong. I frequently am. Let’s examine the facts together." This approach invites constructive dialogue and fosters a positive and collaborative atmosphere without directly confronting Dumbledore.

> *From llama-trace, agent did not seem to call query engine tool to answer question.*

> *Solved - Need to specify resource reference for tool call to be "activated".*

#### Try workflow FunctionAgent package directly. 

In [23]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent
from llama_index.core.workflow import Context

agent = FunctionAgent(tools=tools, llm=Settings.llm)

# context to hold the session/state
ctx = Context(agent)

In [25]:
from llama_index.core.agent.workflow import ToolCallResult, AgentStream

handler = agent.run("Given the principles from the resource: Should Harry Potter tell Dumbledore if he was wrong about something?", ctx=ctx)

async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print(
            f"Call {ev.tool_name} with args {ev.tool_kwargs}\nReturned: {ev.tool_output}"
        )
    elif isinstance(ev, AgentStream):
        print(ev.delta, end="", flush=True)

response = await handler

Call Resource with args {'input': 'Should Harry Potter tell Dumbledore if he was wrong about something?'}
Returned: It would be more effective for Harry Potter to approach Dumbledore with humility and openness. Instead of directly stating that Dumbledore is wrong, Harry could express his thoughts by saying something like, "I may be mistaken, but I have a different perspective on this. Can we discuss it?" This approach fosters a collaborative dialogue and encourages a more receptive atmosphere for discussion.
Based on the principles of effective communication, Harry Potter should approach the situation with humility and openness. Instead of directly stating that Dumbledore is wrong, he could express his thoughts in a way that invites discussion. For example, he might say, "I may be mistaken, but I have a different perspective on this. Can we discuss it?" 

This approach fosters a collaborative dialogue and creates a more receptive atmosphere for discussion, allowing for a constructive e

#### Explore tracing for ReAct Agent

> ***Notes:*** *ReAct Agent differs from Function Calling Agent with its loop - I.e It injects a ReAct prompt (Thought, Action, Observation loop) to LLM input in PrepEvent before emitting the InputEvent.*

In [28]:
class PrepEvent(Event):
    pass


class InputEvent(Event):
    input: list[ChatMessage]


class StreamEvent(Event):
    delta: str


class ToolCallEvent(Event):
    tool_calls: list[ToolSelection]


class FunctionOutputEvent(Event):
    output: ToolOutput

In [29]:
from typing import Any, List

from llama_index.core.agent.react import ReActChatFormatter, ReActOutputParser
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
)
from llama_index.core.llms.llm import LLM
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.tools.types import BaseTool
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.llms.openai import OpenAI


class ReActAgent(Workflow):
    def __init__(
        self,
        *args: Any,
        llm: LLM | None = None,
        tools: list[BaseTool] | None = None,
        extra_context: str | None = None,
        **kwargs: Any,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.tools = tools or []
        self.llm = llm or OpenAI()
        self.formatter = ReActChatFormatter.from_defaults(
            context=extra_context or ""
        )
        self.output_parser = ReActOutputParser()

    @step
    async def new_user_msg(self, ctx: Context, ev: StartEvent) -> PrepEvent:
        # clear sources
        await ctx.store.set("sources", [])

        # init memory if needed
        memory = await ctx.store.get("memory", default=None)
        if not memory:
            memory = ChatMemoryBuffer.from_defaults(llm=self.llm)

        # get user input
        user_input = ev.input
        user_msg = ChatMessage(role="user", content=user_input)
        memory.put(user_msg)

        # clear current reasoning
        await ctx.store.set("current_reasoning", [])

        # set memory
        await ctx.store.set("memory", memory)

        return PrepEvent()

    @step
    async def prepare_chat_history(
        self, ctx: Context, ev: PrepEvent
    ) -> InputEvent:
        # get chat history
        memory = await ctx.store.get("memory")
        chat_history = memory.get()
        current_reasoning = await ctx.store.get(
            "current_reasoning", default=[]
        )

        # format the prompt with react instructions
        llm_input = self.formatter.format(
            self.tools, chat_history, current_reasoning=current_reasoning
        )
        return InputEvent(input=llm_input)

    @step
    async def handle_llm_input(
        self, ctx: Context, ev: InputEvent
    ) -> ToolCallEvent | StopEvent:
        chat_history = ev.input
        current_reasoning = await ctx.store.get(
            "current_reasoning", default=[]
        )
        memory = await ctx.store.get("memory")

        response_gen = await self.llm.astream_chat(chat_history)
        async for response in response_gen:
            ctx.write_event_to_stream(StreamEvent(delta=response.delta or ""))

        try:
            reasoning_step = self.output_parser.parse(response.message.content)
            current_reasoning.append(reasoning_step)

            if reasoning_step.is_done:
                memory.put(
                    ChatMessage(
                        role="assistant", content=reasoning_step.response
                    )
                )
                await ctx.store.set("memory", memory)
                await ctx.store.set("current_reasoning", current_reasoning)

                sources = await ctx.store.get("sources", default=[])

                return StopEvent(
                    result={
                        "response": reasoning_step.response,
                        "sources": [sources],
                        "reasoning": current_reasoning,
                    }
                )
            elif isinstance(reasoning_step, ActionReasoningStep):
                tool_name = reasoning_step.action
                tool_args = reasoning_step.action_input
                return ToolCallEvent(
                    tool_calls=[
                        ToolSelection(
                            tool_id="fake",
                            tool_name=tool_name,
                            tool_kwargs=tool_args,
                        )
                    ]
                )
        except Exception as e:
            current_reasoning.append(
                ObservationReasoningStep(
                    observation=f"There was an error in parsing my reasoning: {e}"
                )
            )
            await ctx.store.set("current_reasoning", current_reasoning)

        # if no tool calls or final response, iterate again
        return PrepEvent()

    @step
    async def handle_tool_calls(
        self, ctx: Context, ev: ToolCallEvent
    ) -> PrepEvent:
        tool_calls = ev.tool_calls
        tools_by_name = {tool.metadata.get_name(): tool for tool in self.tools}
        current_reasoning = await ctx.store.get(
            "current_reasoning", default=[]
        )
        sources = await ctx.store.get("sources", default=[])

        # call tools -- safely!
        for tool_call in tool_calls:
            tool = tools_by_name.get(tool_call.tool_name)
            if not tool:
                current_reasoning.append(
                    ObservationReasoningStep(
                        observation=f"Tool {tool_call.tool_name} does not exist"
                    )
                )
                continue

            try:
                tool_output = tool(**tool_call.tool_kwargs)
                sources.append(tool_output)
                current_reasoning.append(
                    ObservationReasoningStep(observation=tool_output.content)
                )
            except Exception as e:
                current_reasoning.append(
                    ObservationReasoningStep(
                        observation=f"Error calling tool {tool.metadata.get_name()}: {e}"
                    )
                )

        # save new state in context
        await ctx.store.set("sources", sources)
        await ctx.store.set("current_reasoning", current_reasoning)

        # prep the next iteraiton
        return PrepEvent()

#### Run ReAct Agent with Query Engine Tool

In [31]:
# Define agent
react_agent = ReActAgent(
    tools=tools,
    llm = Settings.llm,
    timeout=120,
    verbose = False,
)

handler = react_agent.run(input="With principles from the given resource, how should Harry respond to Dumbledore if he found that the latter said something wrong?")

async for event in handler.stream_events():
    if isinstance(event, StreamEvent):
        print(event.delta, end="", flush=True)

response = await handler

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: Resource
Action Input: {"input": "How should Harry respond to Dumbledore if he found that the latter said something wrong?"}Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Harry should respond to Dumbledore with humility and openness. He could say something like, "I thought differently, but I may be wrong. I often am. Let's examine the facts together." This approach fosters a collaborative discussion rather than confrontation, allowing for a more constructive dialogue.